In [ ]:
# using Pkg
# Pkg.add("JuMP")
# Pkg.add("Gurobi")
# Pkg.add("Distances")
# Pkg.add("Distributions")
# Pkg.add("DataFrames")

In [1]:
using JuMP
using Gurobi
using DataFrames
using Random

# Generate Data

## Constants

In [10]:
warehouse_location_side_size = 2
warehouse_rows = 40
warehouse_columns = 100

time_horizon = 20
num_shipments = 100

max_item_size = 10

S = num_shipments
L = warehouse_rows * warehouse_columns
T = time_horizon

20

In [11]:
Random.seed!(3000)

arrival_location = (0, 1)
departure_location = (warehouse_rows, 0)

distance_to_arrival = zeros(warehouse_rows * warehouse_columns)
for i in 1:warehouse_rows, j in 1:warehouse_columns
    l = (i-1)*warehouse_columns + j
    distance_to_arrival[l] = i * warehouse_location_side_size + j * warehouse_location_side_size
end

distance_to_departure = zeros(warehouse_rows * warehouse_columns)
for i in 1:warehouse_rows, j in 1:warehouse_columns
    l = (i-1)*warehouse_columns + j
    distance_to_departure[l] = ((warehouse_rows - i) * warehouse_location_side_size) + j * warehouse_location_side_size
end

location_max_size = rand(1:max_item_size, warehouse_rows * warehouse_columns)

rand_shipment_data = rand(1:T, (S, 2))
arrivals = minimum(rand_shipment_data, dims=2)
departures = maximum(rand_shipment_data, dims=2)

rand_sizes = rand(1:max_item_size, S)

struct Shipment
    arrival_time::Int
    departure_time::Int
    product_size::Int
end

shipments = Array{Shipment,1}(undef, S)

for s in 1:S
    arrival_time = arrivals[s]
    departure_time = departures[s]
    product_size = rand_sizes[s]

    # Can't arrive and depart at the same time
    if arrival_time == departure_time
        if departure_time == T
            arrival_time -= 1
        else
            departure_time += 1
        end
    end

    shipments[s] = Shipment(arrival_time, departure_time, product_size)
end

# Build Model

In [12]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60);

# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    r[1:S,1:L,1:T] >= 0, Bin
) ;

# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    z[1:S,1:L] >= 0, Bin
) ;

@objective(
    model,
    Min,
    sum(
        (distance_to_arrival[l] + distance_to_departure[l]) / (shipments[s].departure_time - shipments[s].arrival_time + 1) * r[s,l,t]
        for s in 1:S,
            l in 1:L,
            t in 1:T
    )
);

# Within its window, a product is stored in its location and no where else
for s in 1:S
    for l in 1:L
        for t in shipments[s].arrival_time:shipments[s].departure_time
            @constraint(
                model,
                r[s,l,t] == z[s,l]
            );
        end
    end
end

# Each product must be assigned to exactly one location during its time window:
for s in 1:S
    for t in shipments[s].arrival_time:shipments[s].departure_time
        @constraint(
            model,
            sum(
                r[s,l,t]
                for l in 1:L
            ) == 1
        );
    end
end

# Each product is only assigned to a location during its time window:
for s in 1:S
    for t in 1:shipments[s].arrival_time-1
        @constraint(
            model,
            sum(
                r[s,l,t]
                for l in 1:L
            ) == 0
        );
    end
end
for s in 1:S
    for t in shipments[s].departure_time+1:T
        @constraint(
            model,
            sum(
                r[s,l,t]
                for l in 1:L
            ) == 0
        );
    end
end

# Each location can hold at most one product at any specific time:
for l in 1:L
    for t in 1:T
        @constraint(
            model,
            sum(
                r[s,l,t]
                for s in 1:S
            ) <= 1
        );
    end
end

# Each location can only hold items up to max size of the location
for l in 1:L
    for t in 1:T
        for s in 1:S
            @constraint(
                model,
                r[s,l,t] * shipments[s].product_size <= location_max_size[l]
            );
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-08
Set parameter TimeLimit to value 60


In [13]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 120);


# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    z[1:S, 1:L], Bin
) ;


@objective(
    model,
    Min,
    sum(
        (distance_to_arrival[l] + distance_to_departure[l]) * z[s,l]
        for s in 1:S,
            l in 1:L
    )
);


# Each product must be assigned to exactly one location during its time window:
for s in 1:S
    @constraint(
        model,
        sum(
            z[s,l]
            for l in 1:L
        ) == 1
    );
end


# Each location can hold at most one product at any specific time:
for s1 in 1:S
    for s2 in 1:S
        if s1 == s2
            continue
        end

        if shipments[s1].arrival_time <= shipments[s2].departure_time && shipments[s2].arrival_time <= shipments[s1].departure_time
            for l in 1:L
                @constraint(
                    model,
                    z[s1,l] + z[s2,l] <= 1
                );
            end
        end
    end
end

# Each location can only hold items up to max size of the location
for l in 1:L
    for s in 1:S
        @constraint(
            model,
            z[s] * shipments[s].product_size <= location_max_size[l]
        );
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-08
Set parameter TimeLimit to value 120


In [14]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 120);

# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    z[1:S, 1:L], Bin
) ;

@variable(
    model,
    1 <= r[1:S] <= L, Int
) ;

@variable(
    model,
    y[1:S, 1:S], Bin
) ;

@objective(
    model,
    Min,
    sum(
        (distance_to_arrival[l] + distance_to_departure[l]) * z[s,l]
        for s in 1:S,
            l in 1:L
    )
);

# Make r[s] equal to the index of the location we're storing s in
for s in 1:S
    @constraint(
        model,
        sum(
            l * z[s,l]
            for l in 1:L
        ) == r[s]
    );
end

# Each product must be assigned to exactly one location
for s in 1:S
    @constraint(
        model,
        sum(
            z[s,l]
            for l in 1:L
        ) == 1
    );
end


# Each location can hold at most one product at any specific time:
for s1 in 1:S
    for s2 in 1:S
        if s1 == s2
            continue
        end

        if shipments[s1].arrival_time <= shipments[s2].departure_time && shipments[s2].arrival_time <= shipments[s1].departure_time
            @constraint(
                model,
                r[s1] - r[s2] <= -.01 + (L+5) * y[s1,s2]
            );
            @constraint(
                model,
                r[s1] - r[s2] >= .01 - (1-y[s1,s2]) * (L+5)
            );
        end
    end
end

# Each location can only hold items up to max size of the location
for l in 1:L
    for s in 1:S
        @constraint(
            model,
            z[s] * shipments[s].product_size <= location_max_size[l]
        );
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-08
Set parameter TimeLimit to value 120


In [15]:
optimize!(model)

Set parameter TimeLimit to value 120
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 414248 rows, 410100 columns and 1242244 nonzeros
Model fingerprint: 0x2c184f52
Variable types: 0 continuous, 410100 integer (410000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [8e+01, 5e+02]
  Bounds range     [1e+00, 4e+03]
  RHS range        [1e-02, 4e+03]
Presolve removed 400000 rows and 3071 columns
Presolve time: 2.08s
Presolved: 14248 rows, 407029 columns, 841954 nonzeros
Variable types: 0 continuous, 407029 integer (406929 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 71
 AA' NZ     : 1.054e+04
 Factor NZ  : 1.837e+04 (roughly 120 MB of memory)
 Factor Ops : 2.301e+05 (l

In [ ]:
# value.(r[1])

findall(x->x==1, value.(z[1,:]))
# findall(x->x==1, value.(r[1,:,:]))